# Échauffement de Huffman

1. Donner la table des fréquences puis l'arbre de Huffman du texte suivant: `ceci est un exemple de code de Huffman, le construire c'est le maitriser`
2. Utiliser l'arbre de Huffman construit pour coder le texte (ou au moins ses première lettres) et comparer la taille du texte compressé à la taille du texte original en ASCII.

In [ ]:
c:5 e:13 i:4 s:4 t:4 u:3 x:1 p:1 o:2 d:3 h:1 n:3 l:3 f:2 r:4 m:3 a:2 ':1 _:12 ,:1

c: 0010
e: 10
i: 1101
s: 1111
t: 0000
u: 01111
x: 111000
p: 111001
o: 011101
d: 00110
h: 111010
n: 00111
l: 01100
f: 11000
r: 0001
m: 01101
a: 11001
': 111011
_: 010
,: 011100


# Compression LZ77


L'algorithme marche de la façon suivante:

* On a une chaîne de caractères en entrée et on commence en se plaçant sur le premier caractère

* Sur un caractère donné on cherche le plus grand sous-mot qui commence par cette lettre et
  qui apparaît aussi avant. Par exemple si on est en troisième position dans an*a*nas alors le plus long sous-mot 
  est *ana*.
 
* On code ce sous-mot en faisant référence à sa précédente occurrence. Dans l'exemple précédent, 
  le sous-mot fait trois lettres et commence deux lettres avant, on le code par (2,3).

* Si le mot est de taille 0, ou 1, on écrit juste la lettre.


Généralement on borne la fenêtre de recherche dans laquelle on cherche un sous-mot identique afin de limiter
le temps de calcul quand on compresse de grands textes.

1. Donner le résultat de la compression de `aaaaaaaaaaaaaaaaa`.
2. Donner le résultat de la compression de `Chabadi chabada chabababa`.
3. Décompresser le message suivant: `Si, six(4, 2)cies(6, 5)nt(17, 5)cyprès, alor(25, 3)(18, 4)(27, 8)(44, 10)ro(46, 8)(55, 14)`.

In [ ]:
1. 'a(1, 16)'
2. (sans majuscule au premier 'c' et avec des sous mots de taille 2 minimum)
    'chabadi (8, 6)a(8, 6)(2, 4)'
    'cbabbbacbaaabcba'
    'cbab(1, 2)a(7, 3)(1, 2)b(6, 3)'
    '011011101100001100001'
    '011(3, 3)(4, 4)0(1, 3)(6, 7)'
    'acpafeaeafacpeaaapf'
    'acpafea(2, 2)f(10, 3)(6, 2)(1, 2)pf'
3. 'Si, six scies scient six cyprès, alors six cent six scies scieront six cent six cyprès'


# Implémentation

On va compléter le code suivant, pour construire un petit compresseur LZ77.



In [27]:
import tkinter as tk

taille_fenetre = 100 #Taille de la fenêtre de recherche

def match_size(mot,i,j): #renvoie la valeur du plus grand sous-mot commun,dans le mot aux positions position i                             et j avec j < i
    k = 0
    while(i+k < len(mot) and mot[i+k]==mot[j+k]):
        k+=1
    return k

def max_match(mot,i): #renvoie le couple (position,taille) du plus grand match 
                      #trouvé dans mot à partir de la position i
    j = max(0,i-taille_fenetre)  #première position où chercher un match
    max_match = (0,0)
    while j < i: #on cherche un match dans la fenetre de recherche
                 #remplir ici en utilisant la fonction précédente
        taille=match_size(mot, i, j)
        if (max_match[1]<taille):
            max_match=(j, taille)
        j +=1
    return max_match


def compresse():
    texte_a_compresser = entree.get()
    texte_compresse = [] #cette liste doit etre étendue pour contenir le texte compressé
    #construction du code LZ77
    i = 0
    while i < len(texte_a_compresser): #pour chaque lettre du texte
        courant=max_match(texte_a_compresser, i)
        if (courant[1]>=2):
            texte_compresse.append((i-courant[0], courant[1]))
            i+=courant[1]
        else:
            texte_compresse.append(texte_a_compresser[i])
        i+=1
    affichage_compression.config(text = str(texte_compresse)) #affichage du texte compressé 
    resultat.config(text = "Taille compressée de " + str(taille(texte_compresse)))


def taille(liste_LZ):# calcule la taille de la liste. Un caractère compte 1 et 
                                   # une paire d'entier compte 2
    taille = 0
    for elem in liste_LZ:
        if isinstance(elem, str) == True:
            taille += 1
        else:
            taille += 2
    return taille


def change_fenetre():
    global taille_fenetre
    taille_fenetre = int(entree_fenetre.get())


def min_match2():
    global min_match
    min_match = 2


def min_match3():
    global min_match
    min_match = 3


def entier_kbits(n, k):
    res = []
    i = 0
    while i<k:
        if n%2 == 1:
            res.append(1)
        else:
            res.append(0)
        n = n//2
        i+=1
    return res


def code_binaire(texte_compresse):
    res = []
    for elem in texte_compresse:
        if isinstance(elem, str):
            res.append(1)
            res.extend(entier_kbits(ord(elem), 7))
        else:
            res.append(0)
            res.extend(entier_kbits(elem[0], 10))
            res.extend(entier_kbits(elem[1], 5))
    return res


def decompresse(texte_compresse):
    res = []
    for i in range(len(texte_compresse)):
        if isinstance(texte_compresse[i], str):
            res.append(texte_compresse[i])
        else:
            (decalage, taille) = texte_compresse[i]
            pos = len(res)
            for j in range(taille):
                res.append(res[pos - decalage + j])
    return "".join(res)



racine = tk.Tk()
racine.title("Compression de texte")

entree = tk.Entry(racine, width = 50, font = ("helvetica", "20"))
entree.grid(row = 3, column = 0)

entree_fenetre = tk.Entry(racine, width=10, font=("helvetica", "20"))
entree_fenetre.grid(row=1, column=0, columnspan=2)

bouton_fenetre = tk.Button(racine, text="Régler la fenêtre de recherche", command=change_fenetre)
bouton_fenetre.grid(row=2, column=0, columnspan=2)

affichage_compression = tk.Message(racine, font = ("helvetica", "20"), width = 1000)
affichage_compression.grid(row = 5, column = 0, columnspan = 2)

bouton_compresser = tk.Button(racine, text = "Compresser", command = compresse, font = ("helvetica", "30"))
bouton_compresser.grid(row = 4, column = 1)

resultat = tk.Label(racine, font = ("helvetica", "20"))
resultat.grid(row = 6, column = 0, columnspan = 2)

label_taille = tk.Label(racine, text="Taille fenêtre : " + str(taille_fenetre))
label_taille.grid(row=0, column=0, columnspan=2)

bouton_minmatch2 = tk.Button(racine, text="Match minimal 2", command=min_match2)
bouton_minmatch2.grid(row=7, column=0, columnspan=2)

bouton_minmatch3 = tk.Button(racine, text="Match minimal 3", command=min_match3)
bouton_minmatch3.grid(row=8, column=0, columnspan=2)

racine.mainloop()


1. Compléter la fonction `match_size(mot,i,j)` qui trouve le plus grand sous-mot commun aux positions `i` et `j`
   de la chaîne de caractère `mot`. On appelle un tel sous mot un *match*.
   
2. Compléter la fonction `max_match(mot,i)` qui trouve le plus grand match pour la position `i` de la chaîne de caractère `mot`. La fonction renvoie une paire d'entiers qui contient la position par rapport à `i` de ce match
   et sa taille.
3. Compléter la fonction `compresse` qui crée une liste de caratères et de paires d'entiers correspondant au code LZ77 du texte entrée, se trouvant dans la variable `texte_a_compresser`.
4. Compléter la fonction `taille` qui calcule la taille du texte compressé, en comptant 1 pour les lettres et 2 pour les paires d'entiers. Pour distinguer un caractère d'une paire d'entiers dans la liste, on pourra utiliser
   `isinstance(elem,str)` qui est vrai si et seulement si elem est une chaîne de caractère (les caractères en python sont des chaînes de caractère de taille 1).
5. Ajouter un label ou plusieurs labels pour afficher les informations suivantes: la taille de la fenêtre de recherche, la taille du texte original et le rapport de compression (taille/taille_compressée).
6. Permettre à l'utilisateur de changer la taille de la fenêtre de recherche. Que se passe-t-il quand on change sa valeur.
7. Que se passe-t-il quand on interdit les matchs de taille 2 ? Et les matchs de taille 3 ?
8. Est-ce que l'algorithme proposé trouve un code LZ optimal en terme de taille.
9. Proposer un codage binaire d'un code LZ77 et écrire le code qui affiche un tel code binaire.
   On supposera l'entrée donnée en ASCI (sur 7 bits). On pourra coder les entiers ainsi que les caractères sur 8 bits. Attention, il faut différentier les caractères et les paires d'entiers.
